# Giới thiệu
Mercari Price suggestion là cuộc thi dự đoán một mặt hàng- thực sự đáng giá bao nhiêu. Chi tiết nhỏ có thể có nghĩa là sự khác biệt lớn về giá cả. Ví dụ, một trong những chiếc áo len có giá 335 đô la và chiếc còn lại có giá 9,99 đô la thì điều gì quyết định về giá của sản phẩm </br>

Việc định giá sản phẩm thậm chí còn khó hơn trên quy mô lớn, chỉ cần xem xét có bao nhiêu sản phẩm được bán trực tuyến. Quần áo có xu hướng định giá theo mùa mạnh mẽ và bị ảnh hưởng nhiều bởi thương hiệu, trong khi đồ điện tử có giá dao động dựa trên thông số kỹ thuật của sản phẩm. </br>

Mercari muốn đưa ra đề xuất về giá cho người bán, nhưng điều này rất khó vì người bán của họ được phép đưa bất kỳ thứ gì hoặc bất kỳ gói nào lên thị trường của Mercari.

# Mục tiêu
Mục tiêu ở đây là phát triển một hệ thống tự động dự đoán giá của một sản phẩm một cách phù hợp dựa trên thông tin về sản phẩm đó được cung cấp

# Giải nén đầu vào

In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

# Thư viện
import một số thư viện để sử dụng

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.linear_model import Ridge, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer

# 1. Đọc dữ liệu

In [ ]:
#Đọc dữ liệu từ 2 tập 'train.tsv' và tập 'test_stg2.tsv'(đưa về dạng bảng)
train = pd.read_table('train.tsv')
test = pd.read_table('test_stg2.tsv')
print(train.shape)
print(test.shape)

In [ ]:
#Kiểm tra thông tin của tập 'train'
train.info()
train.head()

**Quan sát:**
- Ở tập train, mỗi 1 sản phấm sẽ có các thuộc tính: train_id, name, item_condition_id, category_name, shipping, item_description và cột price
- Ở cột brand_name có nhiều giá trị NaN(giá trị rỗng)

=> kiểm tra các giá trị trong tập 'train'

# 2. Xử lí và phân tích dữ liệu
* **Mục tiêu**: Ở phần này chúng ta sẽ xử lí các dữ liệu. Vì bài toán này là bài toán hồi quy tuyến tính nên cần chuyển các thuộc tính về dạng vector hoặc số thực

* Như đã quan sát ở trên ta thấy rằng ở tập 'train' và tập 'test' đều chứa các giá trị rỗng nên ta cần phải kiểm tra các giá trị rỗng này

> # a. Quan sát tổng quan

In [ ]:
#sử dung hàm isnull() đếm các giá trị null
train.isnull().sum()

=> tồn tại giá trị null ở category_name và brand_name


In [ ]:
train.head()

**=> Quan sát dữ liệu ở trên ta thấy được:**
- Ở tập dữ liệu train và test, cột category_name có nhiều cấp được phân tách bằng "/"
- Có 2 tập có giá trị null
- brand_name có gần một nửa giá trị bị thiếu cả trong tập train và tập test, vì vậy không thể xóa những hàng mà phải lấp các giá trị đó
- trong khi cột category_name thiếu 0,43-0,44% giá trị nên có thể xóa các hàng đó hoặc lấp đầy các giá trị đó


> # b. Xử lí các giá trị rỗng
Việc tồn trại các giá trị rỗng sẽ làm cho đánh giá không chính xác
**Lấp các giá trị null**

In [ ]:
def handle_missing(dataset):
    #xử lý dữ liệu trống
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

In [ ]:
train = handle_missing(train)
test = handle_missing(test)

In [ ]:
#Quan sát dữ liệu ở tập tain sau khi lấp các giá trị null
train.head(10)

**Quan sát:**

- Quan sát 10 sản phẩm đau tiên của tập 'train' các giá trị rống đã được thay thế bằng các text phù hợp
**Sử dụng LabelBinarizer để xử lí các dữ liệu dạng label và Vectorizer để xử lí các dữ liệu dạng text**

Vì đây là mô hình hồi quy nên cần phải đưa dữ liệu về dạng vecto để xử lí

> # c. Xử lí dữ liệu ở cột category_name

In [ ]:
#kiểm tra category_name
train.head(10)
test.head(10)

**Quan sát**
- category_name có 3 mục được phân biệt bằng "/"
=> Chia category_name thành 3 cột riêng biệt
**Chia cột category_name thành subcat_1, subcat_2, subcat_3**

=> việc này giúp cho mô hình huấn luyện chính xác hơn

In [ ]:
#Hàm split_cat dùng để cắt các nhãn trong cột 'category_name' thành các cột riêng biệt
def transform_category_name(category_name):
    try:
        sub1, sub2, sub3 = category_name.split('/')
        return sub1, sub2, sub3
    except:
        return "none", "none", "none"

train['subcat_1'], train['subcat_2'], train['subcat_3'] = zip(*train['category_name'].apply(transform_category_name))
test['subcat_1'], test['subcat_2'], test['subcat_3'] = zip(*test['category_name'].apply(transform_category_name))
train.head(10)

In [ ]:
print("There are %d unique subcat_1." % train['subcat_1'].nunique())
print("There are %d unique subcat_2." % train['subcat_2'].nunique())
print("There are %d unique subcat_3." % train['subcat_3'].nunique())

In [ ]:
train.head()

=> dữ liệu trong tập 'train' có thêm 3 cột 'subcat_1', 'subcat_2', 'subcat_3'

> # d. Quan sát và phân tích các dữ liệu dạng số
- Các cột thuộc dữ liệu dạng số bao gồm: train_id, item_condition_id, price, shipping



In [ ]:
#Định dạng các dữ liệu ở dạng số
pd.set_option('display.float_format', lambda x: '%.5f' % x)
#Thống kê 
train.describe()

**Quan sát**
- Ở đây chỉ có 2 cột dữ liệu dùng để phân tích dự đoán là cột item_condition_id và shipping
- Giá trị của các mặt hàng nhỏ nhất là không tức là có những sản phẩm có thể xem như là miễn phí và lớn nhất là 2009
- Giá trị phổ biến của các sản phẩm dao động từ 17 $ - 2009 $
- Phần lớn các mặt hàng có shipping = 1
> #  **Đánh giá cột price**

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(14,8))
ax.hist(train.price,bins = 30, range = [0,200],label="Price")
ax.set_xlabel('Price',fontsize=15)
ax.set_ylabel('No of items', fontsize=15)
plt.show()

- chúng ta có thể thấy rằng phân phối trên PRICE bị lệch trái

In [ ]:
train["logPrice"] = np.log(train["price"]+1)

from scipy.stats import norm
import seaborn as sns

fig, axes = plt.subplots(1, 3, figsize=(10,4))

axes[0].set_title('Price')
sns.distplot(train.price, ax=axes[0], kde=False)
axes[0].grid()

axes[1].set_title('Price < 75')
sns.distplot(train.price[train.price<75], ax=axes[1], kde=False)
axes[1].grid()

axes[2].set_title('log(Price + 1)')
sns.distplot(train["logPrice"], ax=axes[2], fit=norm, kde=False)
axes[2].set_xticks(range(0,9))
axes[2].grid()

**Quan sát**
- Phân bố của log(price) ít lệch hơn và được phân bố tốt xung quanh giá trị trung bình
=> Sử dụng log(price) thay vì price để huấn luyện
> #  **Đánh giá cột item_condition_id**

In [ ]:
#Biểu đồ phân tích cột item_condition_id
ax = sns.countplot(x = 'item_condition_id',data=train, palette ='Blues_r')
ax.set_title("Tổng số lượng sản phẩm theo item_condition_id", fontsize = 13)

**Quan sát**
- Có 5 loại tình item_condition_id
- Loại 1, loại 2 và loại 3 là phổ biến nhất
- Loại 4 và loại 5 là ít phổ biến nhất
> #  **Đánh giá cột shipping**

In [ ]:
(train['shipping'].value_counts())
def plot_distribution_and_violin(variable):
    fig, axes = plt.subplots(2,1,figsize=(5,6), sharex=True)
    axes[0].set_title(variable)
    sns.countplot(x=variable, data=train, palette="ch:.25", color="c", ax=axes[0])
    sns.violinplot(x=variable, y='logPrice', palette="ch:.25", data=train, ax=axes[1])
    fig.tight_layout()

plot_distribution_and_violin('shipping')

- Giá trị trung bình của những sản phẩm có shipping = 0 nhỏ hơn giá trị trung bình của những sản phẩm có shipping = 1.=> Có khoảng 55% khách hàng không phải trả phí vận chuyển
> #  **Đánh giá cột brand_name**

In [ ]:
train['category_name'].value_counts()[:10]

**Quan sát:**
- Phần lớn sản phẩm không có thương hiệu.
- Ngoài ra top các thương hiệu phổ biến nhất là: PINK, Nike, Victoria's Secret, LuLaRoe, Apple, FOREVER 21, Nintendo, Lululemon, Michael Kors.
> #  **Đánh giá các cột subcat_1 subcat_2 và subcat_3**
> **subcat_1**

In [ ]:
print("Có %d nhãn ở cột subcat_1." % train['subcat_1'].nunique())

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot


x = train['subcat_1'].value_counts().index.values.astype('str')
y = train['subcat_1'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='subcat_1'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)

=> Woman và Beauty là 2 danh mục hàng đầu và chiếm lượng rất lớn
> **subcat_2**

In [ ]:
print("Có %d nhãn ở cột subcat_2." % train['subcat_2'].nunique())

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot


x = train['subcat_2'].value_counts().index.values.astype('str')
y = train['subcat_2'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='subcat_2'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)

> **subcat_3**

In [ ]:
print("Có %d nhãn ở cột subcat_3." % train['subcat_3'].nunique())

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot


x = train['subcat_3'].value_counts().index.values.astype('str')
y = train['subcat_3'].value_counts().values
pct = [("%.2f"%(v*100))+"%"for v in (y/len(train))]

trace1 = go.Bar(x=x, y=y, text=pct)
layout = dict(title= 'Number of Items by Main Category',
              yaxis = dict(title='Count'),
              xaxis = dict(title='subcat_3'))
fig=dict(data=[trace1], layout=layout)
iplot(fig)

Vì các sản phẩm 'women' xuất hiện với số lượng lớn trong Danh mục 'subcat_1', nên các danh mục xuất hiện nhiều nhất trong Danh mục 'subcat_2' phù hợp với danh các danh mục nhiều nhất ở mục 'subcat_2' ('Athletic Apparel','Makeup','Tops&Blueses') và mục 'subcat_3' cũng vậy

> # e. Quan sát dữ liệu ở cột 'item_description'

In [ ]:
from wordcloud import WordCloud
import os
wordcloud = WordCloud(width = 2400, height = 1200).generate(" ".join(train.item_description.astype(str)))
plt.figure(figsize = (13, 10))
plt.imshow(wordcloud)
plt.show()

Các cụm từ "Brand new", "free shipping", "great condition", " good condition", "never worn", "smoke free", " description yet" là những cụm từ xuất hiện nhiều nhất

# 3. Vecto hóa dữ dữ liệu
**Mục tiêu:** là huấn luyện theo mô hình hồi quy tuyến tính nên cần đưa dữ liệu về dạng vecto 

**Sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân**



In [ ]:
lb_item_condition_id = LabelBinarizer(sparse_output=True)
train_condition = lb_item_condition_id.fit_transform(train['item_condition_id'])
test_condition = lb_item_condition_id.transform(test['item_condition_id'])

train_condition.shape

=> ma trận trả về có 1482535 hàng theo 1482535 dữ liệu của từng sản phẩm và có 5 loại item_condition


In [ ]:
lb_shipping = LabelBinarizer(sparse_output=True)
train_shipping = lb_shipping.fit_transform(train['shipping'])
test_shipping = lb_shipping.transform(test['shipping'])

train_shipping.shape

=> ma trận trả về có 1482535 hàng (vì trong tập train có dữ liệu của 1482535 sản phẩm) và có 1 cột vì chỉ có 2 kiểu shipping

In [ ]:
lb_brand_name = LabelBinarizer(sparse_output=True)
train_brand_name= lb_brand_name.fit_transform(train['brand_name'])
test_brand_name = lb_brand_name.transform(test['brand_name'])

train_brand_name.shape

=> ma trận trả về có 1482535 hàng (vì trong tập train có dữ liệu của 1482535 sản phẩm) và có 4810 cột vì  có 4810 thương hiệu


Vì mỗi một chuỗi ở cột "name" khá ngắn nên Sử dụng CountVectorizer() để chuyển cột name từ dạng text về dạng một vecto trên cơ sở số lần xuất hiện của mỗi từ (xuất hiện càng nhiều đánh giá càng cao)

In [ ]:
count_vec = CountVectorizer()

train_name = count_vec.fit_transform(train['name'])
test_name = count_vec.transform(test['name'])

print(train_name.shape)

=> ma trận 'train_name'có 1482535 hàng là vì có 1482535 sản phẩm trong tập 'train' và có 105757 tức là có tất cả 105757 từ xuất hiện trong cột 'name'

Do số lượng từ ở cột item_description nhiều

Sẽ khó khăn hơn khi phân tích mục description vì đó là dữ liệu phi cấu trúc. Nhìn sơ qua về dữ liệu có thể nhận thấy, khi description càng dài thì mặt hàng có xu hướng giá cao hơn. Chúng ta sẽ loại bỏ tất cả các dấu câu, loại bỏ một số từ ngắn (stop words) trong tiếng Anh (như "a", "the", v.v.) và bất kỳ từ nào khác có độ dài nhỏ hơn 3

=> sử dụng TfidfVectorizer() để chuyển đổi dữ liệu thành dạng ma trận

> # TF-IDF (Term Frequency – Inverse Document Frequency)

là 1 kĩ thuật sử dụng trong khai phá dữ liệu văn bản. Trọng số này được sử dụng để đánh giá tầm quan trọng của một từ trong một văn bản.

TF: Term Frequency(Tần suất xuất hiện của từ) là số lần từ xuất hiện trong văn bản. Trong các văn bản thì tần suất xuất hiện của các từ có thể lớn hoặc nhỏ, TF sẽ chia độ dài văn bản (theo số từ)


IDF: Inverse Document Frequency(Nghịch đảo tần suất của văn bản), nhằm đánh giá giá trị của các từ. Khi tính toán TF , tất cả các từ được coi như có độ quan trọng bằng nhau. Nhưng một số từ ngắn như "is" "a" "or"... thường xuất hiện nhiều nhưng giá trị không lớn => IDF sẽ giảm giá trị đánh giá các từ này

In [ ]:
tfidf_des = TfidfVectorizer(max_features=50000, ngram_range=(1, 3), stop_words='english')

train_des = tfidf_des.fit_transform(train['item_description'])
test_des = tfidf_des.transform(test['item_description'])

train_des.shape

Sau khi chuẩn hóa dữ liệu ở các cột category_name, sử dụng LabelBinarizer để chuyển đổi các nhãn nhiều lớp sang nhãn nhị phân ở các cột subcat_1, subcat_2, subcat_3


In [ ]:
lb_cat_1 = LabelBinarizer(sparse_output=True)
train_cat_1 = lb_cat_1.fit_transform(train['subcat_1'])
test_cat_1 = lb_cat_1.transform(test['subcat_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
train_cat_2 = lb_cat_2.fit_transform(train['subcat_2'])
test_cat_2 = lb_cat_2.transform(test['subcat_2'])

lb_cat_3 = LabelBinarizer(sparse_output=True)
train_cat_3 = lb_cat_3.fit_transform(train['subcat_3'])
test_cat_3 = lb_cat_3.transform(test['subcat_3'])

# 4. Mô hình hóa dữ liệu
 **Tạo sparse matrix để kết hợp các dữ liệu với nhau**

In [ ]:
from scipy.sparse import hstack
import gc

sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)

X_train = hstack(sparse_matrix_list).tocsr()
print( X_train.shape)

del X_train
gc.collect()

# 5. Đánh giá mô hình

> # a. Số liệu đánh giá
**RMSLE**

- Lỗi trung bình bình phương (RMSE) là độ lệch chuẩn của phần dư ( lỗi dự đoán ). ... Nó thực hiện điều này bằng cách đo sự khác biệt giữa các giá trị dự đoán và giá trị thực tế . R-MSE càng nhỏ tức là sai số càng bé thì mức độ ước lượng cho thấy độ tin cậy của mô hình có thể đạt cao nhất.
- Công thức:

![alt](https://secureservercdn.net/160.153.137.16/70j.58d.myftpupload.com/wp-content/uploads/2019/03/rmsle-2.png)

Trong đó: 
+ y^i là giá trị ước lượng
+ yi là biến độc lập
+ n=(N – k – 1)
+ N : số tổng lượng quan sát
+ K : tổng lượng biến


In [ ]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [ ]:
from sklearn.model_selection import train_test_split

def run_model(model, matrix_list):
    #X= train[['item_condition_id','shipping','name','brand_name','cat_1','cat_2','cat_3']]
    #X_train, x_test, Y_train, y_test = train_test_split(X, train['price'], test_size=0.25)
    X = hstack(matrix_list).tocsr()
    X_train, x_test, Y_train, y_test = train_test_split(X, np.log1p(train['price']), test_size=0.2)
    model.fit(X_train, Y_train)
    preds = model.predict(x_test)
    del X, X_train, x_test, Y_train
    gc.collect()
    
    return preds, y_test

 > # b Model: Ridge Regression
 
 **Tổng quan:**

- Hồi quy Ridge là một kỹ thuật để phân tích nhiều dữ liệu hồi quy chịu sự đa hình. Khi đa cộng tuyến xảy ra, ước tính bình phương tối thiểu là không thiên vị, nhưng phương sai của chúng lớn nên chúng có thể cách xa giá trị thực. Bằng cách thêm một mức độ sai lệch cho các ước tính hồi quy, hồi quy sườn giúp giảm các lỗi tiêu chuẩn.

- Ridge Regression là mô hình hồi quy phân tích mối quan hệ giữa các biến độc lập và các biến phụ thuộc sử dụng phương pháp Regularization, điều chình mô hình sao cho giảm thiểu các vấn đề Overfitting, tối ưu hay kiểm soát mức độ phức tạp của mô hình để cân đối giữa Biased và Variance Quan đó giảm sai số của mô hình.
- Công thức tổng quát của mô hình:

 >![alt](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQbpyppaTUqoiV9l9SM72xpWq_d31m4-Ofbig&usqp=CAU)

Hệ số λ hay còn gọi là tham số Regularization, là số luôn dương, là giá trị mà ở đó phương trình tuyến tính sẽ tính toán được để sai số của mô hình được giảm tối đa, nghĩa là giá trị λ nào làm cho MSE (ở đây dùng RLMSE) nhỏ nhất thì mô hình đó được chọn

In [ ]:
from sklearn.linear_model import Ridge
model = ridge = Ridge()
Ridge_preds, y_test = run_model(model, matrix_list=sparse_matrix_list)

print("rmsle: "+str(rmsle(np.expm1(y_test), np.expm1(Ridge_preds))))

# 6 Kết quả


In [ ]:
sparse_matrix_list = (train_name, train_des, train_brand_name, train_condition,
                      train_shipping, train_cat_1, train_cat_2, train_cat_3)
X_train = hstack(sparse_matrix_list).tocsr()
X_train

y_train = np.log1p(train['price'])
y_train

Ridge_model =  Ridge()
Ridge_model.fit(X_train, y_train)

sparse_matrix_list = (test_name, test_des, test_brand_name, test_condition,
                      test_shipping, test_cat_1, test_cat_2, test_cat_3)
X_test = hstack(sparse_matrix_list).tocsr()

preds = Ridge_model.predict(X_test)
preds
preds = np.expm1(preds)
preds

In [ ]:
submission = pd.read_csv('sample_submission_stg2.csv')
submission
submission.loc[:, 'price'] = preds
submission

In [ ]:
submission.to_csv('submission.csv', index=False)